In [19]:
# %pip install pygame
import copy 
import pygame
import threading
import time
import heapq

# Visualize the game state

In [2]:
# Define puzzle board dimensions and colors
WINDOW_SIZE = 300
GRID_SIZE = 3
GRID_WIDTH = WINDOW_SIZE // GRID_SIZE
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

current_puzzle = None

def draw_grid(screen):
    for i in range(1, GRID_SIZE):
        pygame.draw.line(screen, BLACK, (i * GRID_WIDTH, 0), (i * GRID_WIDTH, WINDOW_SIZE))
        pygame.draw.line(screen, BLACK, (0, i * GRID_WIDTH), (WINDOW_SIZE, i * GRID_WIDTH))


def draw_puzzle(screen, puzzle):
    font = pygame.font.Font(None, 36)
    for i in range(GRID_SIZE * GRID_SIZE):
        cell_value = puzzle[i]
        if cell_value != 0:
            row = i // GRID_SIZE
            col = i % GRID_SIZE
            cell_text = font.render(str(cell_value), True, BLACK)
            cell_rect = cell_text.get_rect(center=(col * GRID_WIDTH + GRID_WIDTH // 2, row * GRID_WIDTH + GRID_WIDTH // 2))
            screen.blit(cell_text, cell_rect)


def visualize_puzzle(puzzle):
    # if a node is sent instead of a state get hold of the state instead
    if isinstance(puzzle, Node):
        puzzle = puzzle.get_state()
        
    global current_puzzle
    current_puzzle = puzzle

def visualize_path(path):
    for puzzle in path:
        visualize_puzzle(puzzle)
        time.sleep(1)

def puzzle_thread():
    pygame.init()
    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
    pygame.display.set_caption('8-Puzzle Visualization')

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        if current_puzzle is not None:
            screen.fill(WHITE)
            draw_grid(screen)
            draw_puzzle(screen, current_puzzle)
            pygame.display.flip()

def start_puzzle_thread():
    thread = threading.Thread(target=puzzle_thread)
    thread.daemon = True
    thread.start()

def stop_puzzle_thread():
    pygame.quit()

start_puzzle_thread()

# Check if a puzzle is solvable

In [3]:
def is_solvable(puzzle):
    inversions = 0
    for i in range(len(puzzle)):
        for j in range(i + 1, len(puzzle)):
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1
    return inversions % 2 == 0

In [4]:
# Test is_solvable function
puzzle1 = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle2 = [1, 2, 3, 4, 5, 6, 8, 7, 0]

print("Puzzle 1 is solvable:", is_solvable(puzzle1))  # Output: True
print("Puzzle 2 is solvable:", is_solvable(puzzle2))  # Output: False


Puzzle 1 is solvable: True
Puzzle 2 is solvable: False


# Graph implementation

In [5]:
def myPrint(string):
    pass

In [6]:
class Node:
    
    GRID_SIZE = 3
    
    def __init__(self, state):
        self.__state = copy.deepcopy(state)
        self.__parent = None
        self.__path = []
        self.__children = []
        self.__generate_children()            
    

    # Generates all possible moves
    def __generate_children(self):
        self.__children = []
        zero_index = self.__state.index(0)
        row, col = zero_index // self.GRID_SIZE, zero_index % self.GRID_SIZE

        possible_moves = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]

        for move in possible_moves:
            new_row, new_col = move
            if 0 <= new_row < self.GRID_SIZE and 0 <= new_col < self.GRID_SIZE:
                new_state = self.swap(zero_index, new_row * self.GRID_SIZE + new_col)
                self.__children.append(new_state)
        return self.__children

    # Swaps two elements in the state array
    def swap(self, idx1, idx2):
        new_state = self.__state[:]
        new_state[idx1], new_state[idx2] = new_state[idx2], new_state[idx1]
        return new_state

   
    def __str__(self):
        return str(self.__state)    
        
    def __repr__(self):
        return str(self.__state)
    
    def set_parent(self, parent):
        self.__parent = parent
        self.__path = parent.get_path()
        self.__path.append(parent)
    
    def get_path(self):
        return copy.copy(self.__path)
    
    def get_state(self):
        return copy.deepcopy(self.__state)
    
    def get_children(self):
        return copy.deepcopy(self.__children)
    
    def get_parent(self):
        return self.__parent

In [7]:
# Example usage:
initial_state = [1, 2, 5, 3, 4, 8, 6, 7, 0]
puzzle = Node(initial_state)
children = puzzle._Node__generate_children()
print(children)
visualize_puzzle(puzzle)

[[1, 2, 5, 3, 4, 0, 6, 7, 8], [1, 2, 5, 3, 4, 8, 6, 0, 7]]


# BFS

In [8]:
def BFS(node, target, max_nodes=15000):
    if not is_solvable(node.get_state()):
        print("NOT SOLVABLE")
        return [], 0
    
    queue = [node]
    visited = []
    while queue:
        current_node = queue.pop(0)
        current_state = current_node.get_state()
        visited.append(current_state)
        
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path, len(visited)
        
        if len(visited) % 1000 == 0:
            myPrint(f'Visited nodes: {len(visited)}, Queue size: {len(queue)}')
            if len(visited) >= max_nodes:
                print("MAX LIMIT")
                return [], 0
        
        for child_state in current_node.get_children():
            if child_state not in visited:
                child_node = Node(child_state)
                child_node.set_parent(current_node)
                queue.append(child_node)
    return [], 0


In [9]:
target = [0, 1, 2, 3, 4, 5, 6, 7, 8]

puzzle1 = [1, 2, 5, 3, 4, 8, 6, 7, 0]

node = Node(puzzle1)
visualize_puzzle(node)

In [10]:
visualize_puzzle(node)
t1 = time.time()
path, nodes_visited = BFS(node, target, max_nodes=100000)
t2 = time.time()
time_taken = t2 - t1
f'visited nodes: {nodes_visited}, path length: {len(path)}, time taken: {time_taken}'

'visited nodes: 17, path length: 5, time taken: 0.0005037784576416016'

In [11]:
visualize_path(path)

# DFS

In [12]:
def DFS(node, target, max_nodes=10000):
    if not is_solvable(node.get_state()):
        print("NOT SOLVABLE")
        return [], 0
    
    stack = [node]
    visited = []
    while stack:
        current_node = stack.pop()
        current_state = current_node.get_state()
        visited.append(current_state)
        
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path, len(visited)
        
        if len(visited) % 1000 == 0:
            myPrint(f'visited nodes: {len(visited)}, stack size: {len(stack)}')
            if len(visited) >= max_nodes:
                print("MAX LIMIT")
                return [], 0
            
        for child in current_node.get_children():
            if child not in visited:
                child = Node(child)
                child.set_parent(current_node)
                stack.append(child)
    return [], 0

In [15]:
target = [0, 1, 2, 3, 4, 5, 6, 7, 8]

puzzle1 = [1, 2, 5, 3, 4, 8, 6, 7, 0]

node = Node(puzzle1)
visualize_puzzle(node)

In [16]:
t1 = time.time()
path, nodes_visited = DFS(node, target, max_nodes=1000000)
t2 = time.time()
time_taken = t2 - t1

f'visited nodes: {nodes_visited}, path length: {len(path)}, time taken: {time_taken}'


'visited nodes: 33, path length: 33, time taken: 0.0010135173797607422'

In [17]:
visualize_path(path)

# A*

In [21]:
def manhatten_estimate_cost(state):
    cost = 0
    for i in range(len(state)):
        value = state[i]
        if value != 0:
            current_row, current_col = i // 3, i % 3
            target_row, target_col = (value - 1) // 3, (value - 1) % 3
            cost += abs(current_row - target_row) + abs(current_col - target_col)
    return cost

In [22]:
def euclidean_estimate_cost(state):
    cost = 0
    for i in range(len(state)):
        if state[i] != 0:
            current_row, current_col = i // 3, i % 3
            target_row, target_col = (state[i] - 1) // 3, (state[i] - 1) % 3
            cost += ((current_row - target_row) ** 2 + (current_col - target_col) ** 2) ** 0.5
    return cost

In [23]:
def a_star(node, target, estimate_function=manhatten_estimate_cost, max_nodes=100000):
    if not is_solvable(node.get_state()):
        return None
    
    # Q values are tuples (weight, insert_order, node). insert order is used to break ties
    queue = [(0, 0, node)]
    
    # Q inserts counter
    i = 1
    
    visited = []
    while queue:
        
        # pop the node with the lowest weight
        current_node = heapq.heappop(queue)[-1]
        current_state = current_node.get_state()
        visited.append(current_state)
        
        # check if goal state is reached
        if current_state == target:
            final_path = current_node.get_path()
            final_path.append(current_node)
            return final_path, len(visited)
        
        if len(visited) % 1000 == 0:
            myPrint(f'visited nodes: {len(visited)}, queue size: {len(queue)}')
            if len(visited) >= max_nodes:
                print("max nodes reached")
                return None
        
        for child in current_node.get_children():
            if child not in visited:
                child = Node(child)
                child.set_parent(current_node)
                
                # calculate weight by adding the manhatten distance to goal and the path length
                h_n = len(child.get_path())
                g_n = estimate_function(child.get_state())
                weight = h_n + g_n
                
                # push to priority queue
                heapq.heappush(queue, (weight, i, child))
                i+=1
    return None

In [24]:
target = [0, 1, 2, 3, 4, 5, 6, 7, 8]

puzzle1 = [1, 2, 5, 3, 4, 8, 6, 7, 0]

node = Node(puzzle1)
visualize_puzzle(node)

## Manhatten

In [25]:
t = time.time()
path, visited_nodes = a_star(node, target, estimate_function=manhatten_estimate_cost)
t2 = time.time()
time_taken = t2 - t
# visualize_path(path)
f'visited nodes: {visited_nodes}, path length: {len(path)}, time taken: {time_taken}' 

'visited nodes: 16, path length: 5, time taken: 0.0009982585906982422'

In [26]:
visualize_path(path)

## Euclidean

In [27]:
t = time.time()
path, visited_nodes = a_star(node, target, estimate_function=euclidean_estimate_cost)
t2 = time.time()
time_taken = t2 - t

# visualize_path(path)
f'visited nodes: {visited_nodes}, path length: {len(path)}, time taken: {time_taken}' 

'visited nodes: 37, path length: 5, time taken: 0.002010822296142578'

In [28]:
visualize_path(path)

# Benchmarking

In [29]:
def manhatten_A_star(path, target):
    return a_star(path, target, estimate_function=manhatten_estimate_cost)
    
def euclidean_A_star(path, target):
    return a_star(path, target, estimate_function=euclidean_estimate_cost)
    

In [30]:
def time_algorithms(node, target):
    algorithms = [BFS, DFS, manhatten_A_star, euclidean_A_star]
    times = [[] for _ in range(len(algorithms))]
    for i in range(len(algorithms)):
        for _ in range(10):
            t = time.time()
            ret = algorithms[i](node, target)
            t2 = time.time()
            time_taken = t2 - t
            if time_taken > 60 or ret == None:
                times[i] = [-1]
                break
            times[i].append(time_taken)
            
    avg_times = [sum(time) / len(time) for time in times]
    return avg_times

In [31]:
target = [0, 1, 2, 3, 4, 5, 6, 7, 8]

puzzle1 = [1, 2, 5, 3, 4, 8, 6, 7, 0]

node = Node(puzzle1)
visualize_puzzle(node)

In [32]:
t = time_algorithms(node, target)
t

[0.0005019187927246093,
 0.0011504173278808593,
 0.0010246038436889648,
 0.0018053531646728515]